In [1]:
import numpy as np
import os
CurrentPath = os.getcwd()
data_1 = np.loadtxt(CurrentPath + '/2025/data_1.txt', delimiter=' ')
X = data_1.T[0];Y = data_1.T[1]; var = data_1.T[2]

Metropolis-Hastings algorithm: 
1. Propose a prior

In [2]:
# 0 < a < 1, sigma_a = 0.0005
# -2.6 < b < -2.4, sigma_b = 0.1
# 6.7 < c < 6.9, sigma_c = 0.1
# 16 < d < 18 sigma_d = 1
from numpy import random
a_init = random.uniform(0,1)
b_init = random.uniform(-2.6,  -2.4)
c_init = random.uniform(6.7, 6.9)
d_init = random.uniform(16, 18)



2. calculate the likelihood = chisq

In [3]:
def cube(x,a,b,c,d):
    return a*x**3 + b*x**2 + c*x + d

def chisq(x0,y0,var,a,b,c,d):
    diff = (y0 - cube(x0,a,b,c,d))**2
    chisq = np.sum(diff/var)
    return chisq


3. Create a markov chain

In [5]:
def Markov(paramk,paramkp1,x,y,var):
    minuschisqk = -1/chisq(x,y,var,paramk[0],paramk[1],paramk[2],paramk[3])/2
    minuschisqkp1 = -1/chisq(x,y,var,paramkp1[0],paramkp1[1],paramkp1[2],paramkp1[3])/2
    lnr = np.log(np.random.uniform(0.,1.))
    if minuschisqkp1 - minuschisqk > lnr:
        moved = True
    else:
        moved = False
    return paramk, paramkp1, lnr, minuschisqk, minuschisqkp1, moved

# 0 < a < 1, sigma_a = 0.005
# -2.6 < b < -2.4, sigma_b = 0.1
# 6.7 < c < 6.9, sigma_c = 0.1
# 16 < d < 18 sigma_d = 1

def MCMC(paraminit, burn_in, nstep,x,y,var):
    k = 1
    param0 = paraminit
    arr = np.zeros((nstep - burn_in +1,len(param0)*2 + 5))
    arr[0,:] = np.hstack((param0,np.zeros(len(param0) + 4),np.array([k]))) # save zero values : [param0[0],param0[1],param0[2], param0[3], 0,0,0,0,0,0,0,0,k = 0]
    print(arr[0])
    while k<=nstep:
        paramkp1 = np.array([param0[0] + random.normal(0,0.005), param0[1]+ random.normal(0,0.1), param0[2]+random.normal(0,0.1), param0[3]+random.normal(0,1)]) # step size
        print(paramkp1)
        param0, paramkp1, lnr, minuschisqk, minuschisqkp1, moved = Markov(param0, paramkp1,x,y,var)
        if moved == True:
                param0 = paramkp1
        else:
            continue
        if k>burn_in and k<=nstep:
            print(k)
            col = np.hstack((param0, paramkp1, np.array([lnr, minuschisqk, minuschisqkp1, moved, k])))
            arr[k - burn_in,:] = col
            print(col)
            k+=1
            
        print(k)
        k+=1
    return arr

In [12]:
paraminit = np.array([a_init, b_init, c_init, d_init])
param0 = paraminit
paramkp1 = np.array([param0[0] + random.normal(0,0.005), param0[1]+ random.normal(0,0.1), param0[2]+random.normal(0,0.1), param0[3]+random.normal(0,1)]) # step size
k = 1
param0 = paraminit
arr = np.zeros((3 - 1 +1,len(param0)*2 + 5))
arr[0,:] = np.hstack((param0,np.zeros(len(param0) + 4),np.array([k]))) # save zero values : [param0[0],param0[1],param0[2], param0[3], 0,0,0,0,0,0,0,0,k = 0]
print(arr[0])
while k<=3:
    paramkp1 = np.array([param0[0] + random.normal(0,0.005), param0[1]+ random.normal(0,0.1), param0[2]+random.normal(0,0.1), param0[3]+random.normal(0,1)]) # step size
    print(paramkp1)
    param0, paramkp1, lnr, minuschisqk, minuschisqkp1, moved = Markov(param0, paramkp1,X,Y,var)

    if k>1 and k<=3:
        print(k)
        col = np.hstack((param0, paramkp1, np.array([lnr, minuschisqk, minuschisqkp1, moved, k])))
        arr[k - 1,:] = col
        print(col)
        k+=1
    if moved == True:
            param0 = paramkp1
    else:
        continue    
    print(k)
    k+=1


[ 0.04354365 -2.44973114  6.72984597 16.47597506  0.          0.
  0.          0.          0.          0.          0.          0.
  1.        ]
[ 0.04868951 -2.43367328  6.79987096 17.58105752]
1
[ 0.05211711 -2.52840015  6.66277628 16.68869584]
2
[ 4.86895062e-02 -2.43367328e+00  6.79987096e+00  1.75810575e+01
  5.21171146e-02 -2.52840015e+00  6.66277628e+00  1.66886958e+01
 -2.18580931e+00 -1.31691118e-13 -1.32548773e-13  1.00000000e+00
  2.00000000e+00]
3


In [13]:
arr

array([[ 4.35436465e-02, -2.44973114e+00,  6.72984597e+00,
         1.64759751e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 4.86895062e-02, -2.43367328e+00,  6.79987096e+00,
         1.75810575e+01,  5.21171146e-02, -2.52840015e+00,
         6.66277628e+00,  1.66886958e+01, -2.18580931e+00,
        -1.31691118e-13, -1.32548773e-13,  1.00000000e+00,
         2.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [11]:
print(float(True))

1.0
